# Assignment 2
Welcome to the second programming assigment for the course. This assignments will help to familiarise you with entangled states and Boolean function oracles while revisiting the topics discussed in this week's lectures. 

### Submission Guidelines
For final submission, and to ensure that you have no errors in your solution, please use the 'Restart and Run All' option availble in the Kernel menu at the top of the page. 
To submit your solution, run the completed notebook and attach the solved notebook (with results visible) as a file on the course website. 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries and configuring account
import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator
from qiskit.visualization import *
from qiskit.quantum_info import *
success_msg = 'Your answer is correct.'
fail_msg = 'Your answer is not correct. Please try again.'
basis_gates = ['id', 'x', 'y', 'z', 's', 't', 'sdg', 'tdg', 'h', 'p', 'sx' ,'r', 'rx', 'ry', 'rz', 'u', 'u1', 'u2', 'u3', 'cx', 'ccx', 'barrier', 'measure', 'snapshot']

## Entanglement
Multi-partite entangled states are also possible. A very popular example of such a state is the  GHZ state, named after the authors who first proposed it's interesting properties. This belongs to a class of multi-qubit states called _cat states_, after Schrodinger's cat. You will create this state in the first problem.
## **Problem 1**
Prepare the GHZ state $|\text{GHZ}\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle)$ in a quantum circuit **using only the basic single-qubit gates and CNOT**. Below we have provided you with some code to create a quantum circuit. Remember that a qubit in a quantum circuit always begins in the $|0\rangle$ state. Add appropriate gates. A standard basis measurement has already been added for you.

In [ ]:
qc1 = QuantumCircuit(3) 
# Insert gates below to create the state 

# Do not change below this line. You do not need to add an additional measurement. 
qc1.measure_all()
qc1.draw('mpl')

<div class="alert alert-block alert-info"><b>Instructions:</b> Once your circuit is ready, run the cell below to check your answer. </div>

In [ ]:
try:
    assert list(qc1.count_ops()) != [], "Circuit cannot be empty"
    assert set(qc1.count_ops().keys()).difference(basis_gates) == set(), f"Only the following basic gates are allowed: {basis_gates}"
    assert all([type(gate[0]) == qiskit.circuit.measure.Measure for gate in qc1.data[-3:len(qc1.data)]]), "Measurement must be the last operation in a circuit."
    sv_check = Statevector.from_instruction(qc1.remove_final_measurements(inplace=False)).equiv((Statevector.from_label('000')+Statevector.from_label('111'))/np.sqrt(2))
    assert sv_check, "You did not prepare the correct state."
    
    job = execute(qc1, backend=QasmSimulator(), shots=1024, seed_simulator=0)
    counts = job.result().get_counts()
    print(success_msg if (sv_check) else fail_msg)
except AssertionError as e:
    print(f'Your code has an error:  {e.args[0]}')
except Exception as e:
    print(f'This error occured: {e.args[0]}')
plot_histogram(counts)

## A quantum oracle implementation of the classical OR operation
We've already seen that the Toffoli gate implements the quantum version of the classical AND operation. The first part of this exercise will require you to construct such a quantum implementation for the OR operation.
The logical OR operation takes two Boolean inputs and returns 1 as the result if either or both of the inputs are 1. It is often denoted using the $\vee$ symbol (it is also called the disjunction operation). The truth table for the classical OR operation is given below:

| $x$ 	| $y$ 	| $x\vee y$ 	|
|-----	|-----	|-----------	|
| 0   	| 0   	| 0         	|
| 0   	| 1   	| 1         	|
| 1   	| 0   	| 1         	|
| 1   	| 1   	| 1         	|

### De Morgan's laws
Finding a gate that is the direct quantum analogue of the OR operation might prove to be difficult. Luckily, there are a set of two relation in Boolean algebra that can provide a helpful workaround. 
$$\overline{x\vee y} = \overline{x} \wedge \overline{y}$$
This is read as _not ($x$ or $y$) = not $x$ and not $y$_
$$\overline{x\wedge y} = \overline{x} \vee \overline{y}$$
This is read as _not ($x$ or $y$) = not $x$ and not $y$_
## **Problem 2**
1. Using the expressions for De Morgan's laws above, construct a Boolean formula for $x \vee y$ consisting only of the logical AND and NOT operations. 
2. We have provided the `QuantumCircuit()` for a quantum bit oracle to implement the OR operation. Apply the appropriate gates to this circuit based on the expression calculated in Step 1. Do NOT add a measurement

<div class="alert alert-block alert-warning"><b>Warning: </b>Please be careful to ensure that the circuit below matches the oracle structure i.e. the input qubit states are not altered after the operation of the oracle.</div>

In [ ]:
or_oracle = QuantumCircuit(3)
# Insert gates below

# Do not change below this line
or_oracle.draw(output='mpl')

<div class="alert alert-block alert-info"><b>Instructions:</b> Once your circuit is ready, run the cell below to check your answer. </div>

In [ ]:
or_tt = ['000', '011', '101', '111']
def check_or_oracle(tt_row):
    check_qc = QuantumCircuit(3)
    for i in range(2):
        if (tt_row[i] == '1'):
            check_qc.x(i)
    check_qc.extend(or_oracle)
    check_qc.measure_all()
    return (execute(check_qc.reverse_bits(),backend=QasmSimulator(), shots=1).result().get_counts().most_frequent() == tt_row)
try:
    assert list(or_oracle.count_ops()) != [], f"Circuit cannot be empty"
    assert 'measure' not in or_oracle.count_ops(), f"Please remove measurements"
    assert set(or_oracle.count_ops().keys()).difference(basis_gates) == set(), f"Only the following basic gates are allowed: {basis_gates}"
    for tt_row in or_tt:     
        assert check_or_oracle(tt_row), f" Input {tt_row[0:2]}: Your encoding is not correct"
    print("Your oracle construction passed all checks")
except AssertionError as e:
    print(f'Your code has an error:  {e.args[0]}')
    
except Exception as e:
    print(f'This error occured: {e.args[0]}')